In [8]:
%load_ext autoreload
%autoreload 2
from data_api import *

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [13]:
database = database_api('/Users/ziandong/TreeMusicRecommendation/Data/conf.json')
last_fm = lastfm_api('/Users/ziandong/TreeMusicRecommendation/Data/conf.json')
result = last_fm.get_user_friends(username='rj')

In [14]:
print(result)

[{'name': 'Babs_05', 'url': 'https://www.last.fm/user/Babs_05', 'country': 'None', 'playlists': '0', 'playcount': '0', 'image': [{'size': 'small', '#text': 'https://lastfm.freetls.fastly.net/i/u/34s/53595648e87980477107a93ec5021e71.png'}, {'size': 'medium', '#text': 'https://lastfm.freetls.fastly.net/i/u/64s/53595648e87980477107a93ec5021e71.png'}, {'size': 'large', '#text': 'https://lastfm.freetls.fastly.net/i/u/174s/53595648e87980477107a93ec5021e71.png'}, {'size': 'extralarge', '#text': 'https://lastfm.freetls.fastly.net/i/u/300x300/53595648e87980477107a93ec5021e71.png'}], 'registered': {'unixtime': '1131903141', '#text': '2005-11-13 17:32'}, 'realname': 'Badass Mod from the Future - Last Seen Tomorrow Afternoon', 'subscriber': '0', 'bootstrap': '0', 'type': 'mod'}, {'name': 'franhale', 'url': 'https://www.last.fm/user/franhale', 'country': 'United Kingdom', 'playlists': '0', 'playcount': '0', 'image': [{'size': 'small', '#text': 'https://lastfm.freetls.fastly.net/i/u/34s/ccaaaea09ac0

In [15]:
"construrt userdatabse with 100000 users"
import queue
start_username = "rj"
user_queue = queue.Queue()
user_queue.put(start_username)
users_processed = set()
users_processed.add(start_username)

while not user_queue.empty() and len(users_processed) < 10000-145:
    current_username = user_queue.get()
    friends = last_fm.get_user_friends(current_username)

    if friends:
        for friend in friends:
            friend_name = friend['name']
            if friend_name not in users_processed:
                users_processed.add(friend_name)
                user_queue.put(friend_name)
                database.save_users([friend])  # Assuming save_users takes a list
                
                # If we've reached 10,000 users, we can break early
                if len(users_processed) >= 10000:
                    break

print(f"Total users in database: {len(users_processed)}")
